In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

health_df = pd.read_csv('data_source\LengthOfStay.csv')
health_df.set_index("eid", inplace=True)

In [20]:
print(f'There dataset consists of {health_df.shape[0]} rows, and {health_df.shape[1]} columns\n')

There dataset consists of 100000 rows, and 29 columns



In [21]:
health_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100000 entries, 1 to 100000
Data columns (total 29 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   vdate                       100000 non-null  object 
 1   rcount                      100000 non-null  object 
 2   gender                      100000 non-null  object 
 3   dialysisrenalendstage       100000 non-null  int64  
 4   asthma                      100000 non-null  int64  
 5   irondef                     100000 non-null  int64  
 6   pneum                       100000 non-null  int64  
 7   substancedependence         100000 non-null  int64  
 8   psychologicaldisordermajor  100000 non-null  int64  
 9   depress                     100000 non-null  int64  
 10  psychother                  100000 non-null  int64  
 11  fibrosisandother            100000 non-null  int64  
 12  malnutrition                100000 non-null  int64  
 13  hemo               

In [26]:
health_df.describe()[0]

KeyError: 0